In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# class CustomCNN(nn.Module):
#     def __init__(self, num_classes=10, num_conv_layers=3, in_channels=3, hidden_channels=64):
#         super(CustomCNN, self).__init__()
#         self.num_conv_layers = num_conv_layers
#         self.in_channels = in_channels
# 
#         layers = []
#         for i in range(num_conv_layers):
#             layers.append(
#                 nn.Conv2d(
#                     in_channels=in_channels if i == 0 else hidden_channels,
#                     out_channels=hidden_channels,
#                     kernel_size=3,
#                     stride=1,
#                     padding=1
#                 )
#             )
#             layers.append(nn.ReLU())
#             layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
# 
#         self.conv_layers = nn.Sequential(*layers)
#         self.fc1 = nn.Linear(hidden_channels * (32 // (2 ** num_conv_layers)) ** 2, 32)
#         self.fc2 = nn.Linear(32, num_classes)
# 
#     def forward(self, x):
#         x = self.conv_layers(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = nn.ReLU(x)
#         x = self.fc2(x)
#         return x

# # Example usage:
# # Initialize a model with 5 convolutional layers, 3 input channels (e.g., RGB images), and 10 output classes.
# model = CustomCNN(num_classes=3, num_conv_layers=5, in_channels=3, hidden_channels=64)
# 
# # Print the model architecture
# print(model)


In [8]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchvision import transforms

class CustomCSVDataset(Dataset):
    def __init__(self, root_dir, transform=None, fixed_length=1000, included_classes=None):
        self.root_dir = root_dir
        self.transform = transform
        self.fixed_length = fixed_length
        self.data = []
        self.labels = []

        self.root = sorted(os.listdir(root_dir))  # 假设目录名即为类名
        # self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        for subjects_idx, subjects in enumerate(self.root):
            subjects_dir = os.path.join(root_dir, subjects)
            
            if included_classes is not None:
                # for class_index, class_name in enumerate(os.listdir(subjects_dir)):
                for class_index, class_name in enumerate(included_classes):
                        user_dir = os.path.join(subjects_dir, class_name)
                        for csv_file in os.listdir(user_dir):
                            file_path = os.path.join(user_dir, csv_file)
                            # print(file_path)
                            data_frame = pd.read_csv(file_path, index_col=0, header=0, usecols= [0,1,2,3])
                            self.data.append(data_frame.values)
                            self.labels.append(class_index)
                            # print(self.labels)
            else:
                 for class_index, class_name in enumerate(os.listdir(subjects_dir)):
                        user_dir = os.path.join(subjects_dir, class_name)
                        for csv_file in os.listdir(user_dir):
                            file_path = os.path.join(user_dir, csv_file)
                            # print(file_path)
                            data_frame = pd.read_csv(file_path, index_col=0, header=0, usecols= [0,1,2,3])
                            self.data.append(data_frame.values)
                            self.labels.append(class_index)
                            # print(self.labels)
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        # Truncate or pad the sample to fixed_length
        if len(sample) > self.fixed_length:
            sample = sample[:self.fixed_length]
        else:
            padding = np.zeros((self.fixed_length - len(sample), sample.shape[1]))
            sample = np.vstack((sample, padding))

        if self.transform:
            sample = self.transform(sample)
        
        # Reshape sample to (channels, height, width)
        sample = torch.tensor(sample, dtype=torch.float32).permute(1, 0)  # (features, sequence_length)
        sample = sample.unsqueeze(1)  # Add a channel dimension

        return sample, label

class ToTensorAndNormalize:
    def __call__(self, sample):
        # 合并通道
        # sample = np.mean(sample, axis=1, keepdims=True)
        # 标准化
        sample = (sample - sample.mean()) / sample.std()
        return torch.tensor(sample, dtype=torch.float32)

# Instantiate the dataset with transformations
# dataset = CustomCSVDataset(root_dir='datasets/Merge_fold/ByNum_spline5', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t'])
# dataset = CustomCSVDataset(root_dir='datasets/Merge_fold/ByNum_spline5', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t', 'a7t','a8t'])
# # # 
# # # # Create a DataLoader for batch processing
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
# # 
# # # (batch_size, channels, height, width)
# # # Example of iterating through the DataLoader
# for inputs, labels in dataloader:
#     print(labels)
    # print(inputs.shape, labels.shape)



In [9]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(CustomCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=(3, 3), padding=(1, 1)),  # output size: (batch_size, 32, 600, 3)
            # nn.Conv2d(1, 32, kernel_size=(3, 3), padding=(1, 1)),  # output size: (batch_size, 32, 600, 1)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2)),  # output size: (batch_size, 32, 300, 3)
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=(1, 1)),  # output size: (batch_size, 64, 300, 3)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2)),  # output size: (batch_size, 64, 150, 3)
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=(1, 1)),  # output size: (batch_size, 128, 150, 3)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2))  # output size: (batch_size, 128, 75, 3)
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 75 * 1, 512),
            nn.ReLU(),
            # nn.Linear(512,256),
            # nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        # print(x.shape)
        x = torch.flatten(x, 1)
        # print(x.shape)
        x = self.fc(x)
        return x
    


In [10]:
import torch.optim as optim

# Define model, loss function, and optimizer
# model = CustomCNN(num_classes=3, num_conv_layers=5, in_channels=3, hidden_channels=64)

model = CustomCNN(num_classes=3)
criterion = nn.CrossEntropyLoss()

train_dataset0 = CustomCSVDataset(root_dir='datasets/Merge_fold_0713/ByNum_spline3', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t', 'a7t','a8t'])
train_dataset1 = CustomCSVDataset(root_dir='datasets/Merge_fold_0713/ByNum_spline4', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t', 'a7t','a8t'])
train_dataset2 = CustomCSVDataset(root_dir='datasets/Merge_fold_0713/ByNum_spline5', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t', 'a7t','a8t'])

finetune_dataset = CustomCSVDataset(root_dir='datasets/USC-HAD-clean', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t', 'a7t','a8t'])
test_dataset = CustomCSVDataset(root_dir='datasets/USC-HAD-clean', transform=ToTensorAndNormalize(), fixed_length=600, included_classes=['a1t', 'a7t','a8t'])

# Create data loaders
train_dataloaderO = DataLoader(train_dataset0, batch_size=5, shuffle=True)
train_dataloader1 = DataLoader(train_dataset1, batch_size=5, shuffle=True)
train_dataloader2 = DataLoader(train_dataset2, batch_size=5, shuffle=True)
finetune_dataloader = DataLoader(finetune_dataset, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)
# 
# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     for inputs, labels in dataloader:
#         # Zero the parameter gradients
#         optimizer.zero_grad()
# 
#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
# 
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
# 
#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")


In [11]:
# Define optimizer for fine-tuning (only trainable parameters will be optimized)


# Validation function to calculate accuracy
def validate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()
    return correct / total

optimizer = optim.SGD(model.parameters(), lr=0.0005)


num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_dataloaderO:
    # for inputs, labels in finetune_dataloader:
    #     print(inputs.shape)
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    train_accuracy = validate(model, train_dataloaderO)
    test_accuracy = validate(model, test_dataloader)
    print(f"0Pretraining Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
# 
# Pretraining loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     for inputs, labels in train_dataloader1:
#     # for inputs, labels in finetune_dataloader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # print(outputs, labels)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
# 
#     train_accuracy = validate(model, train_dataloader1)
#     test_accuracy = validate(model, test_dataloader)
#     print(f"1Pretraining Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
# 
# num_epochs = 10
# for epoch in range(num_epochs):
#     for inputs, labels in train_dataloader2:
#     # for inputs, labels in finetune_dataloader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         # print(outputs, labels)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
# 
#     train_accuracy = validate(model, train_dataloader2)
#     test_accuracy = validate(model, test_dataloader)
#     print(f"2Pretraining Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")


# # Fine-tuning loop
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in finetune_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    train_accuracy = validate(model, finetune_dataloader)
    test_accuracy = validate(model, test_dataloader)
    print(f"Fine-tuning Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

C:\Users\12820\AppData\Local\Temp\ipykernel_36420\3014035051.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32).permute(1, 0)  # (features, sequence_length)


0Pretraining Epoch 1/10, Loss: 1.0938, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 2/10, Loss: 1.0969, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 3/10, Loss: 1.1077, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 4/10, Loss: 1.1054, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 5/10, Loss: 1.1039, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 6/10, Loss: 1.0975, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 7/10, Loss: 1.1094, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 8/10, Loss: 1.1054, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 9/10, Loss: 1.0796, Train Accuracy: 0.3333, Test Accuracy: 0.3333
0Pretraining Epoch 10/10, Loss: 1.1057, Train Accuracy: 0.3333, Test Accuracy: 0.3333
Fine-tuning Epoch 1/10, Loss: 0.8440, Train Accuracy: 0.8714, Test Accuracy: 0.8714
Fine-tuning Epoch 2/10, Loss: 0.5223, Train Accuracy: 0.9095, Tes